In [3]:
import pyedflib
import numpy as np


In [2]:
PATH = '/project/MASS/SS1_EDF/01-01-0001 Base.edf'
PATH_real = '/project/MASS/SS1_EDF/01-01-0001 PSG.edf'

In [3]:
edf_lib_annotations =  pyedflib.EdfReader(PATH)
edf_lib_real =  pyedflib.EdfReader(PATH_real)


In [4]:
# Extract the sleep stage annotations
annotations = edf_lib_annotations.readAnnotations()
# Extract the signal 
signal = edf_lib_real.readSignal(0)

In [5]:
# Get a list of all the different values in array
unique_values = np.unique(annotations[2])

In [8]:
annotations[0]

array([9.00000000e+00, 3.90000000e+01, 6.90000000e+01, ...,
       3.04590001e+04, 3.04890001e+04, 3.05190001e+04])

In [6]:
unique_values.shape

(6,)

In [52]:
signal.size / 256

30554.0

In [7]:
unique_values

array(['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage ?',
       'Sleep stage R', 'Sleep stage W'], dtype='<U13')

In [68]:
curr_index = 0
labels = []
for annotation in np.stack(annotations).T:
    onset, duration, label = annotation
    # Check if we have some empty space
    # print(f"Before: onset: {onset}, duration: {duration}, label: {label}")
    onset = float(onset)
    duration = float(duration)
    # print(f"After: onset: {onset}, duration: {duration}, label: {label}")

    if onset > curr_index:
        # print(f"Empty space between {curr_index} and {onset}")
        # Add empty labels
        if int(onset - curr_index) > 0:
            print(f"adding empty labels {int(onset - curr_index)}")
            labels += ['?'] * int(onset - curr_index)
    # Add the label
    labels += [label.strip('Sleep stage ')] * int(duration)
    curr_index = onset + duration

print(len(labels))

adding empty labels 9
30549


In [63]:
# Count how many labels we have
np.unique(np.array(labels), return_counts=True)

(array(['1', '2', '3', '?', 'R', 'W'], dtype='<U1'),
 array([ 2070, 15000,  3450,  3009,  5220,  1800]))

In [73]:
array = np.zeros((180, 30500))
# Save array to csv file
np.savetxt("test.csv", array, delimiter=",")

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [3]:
import random
sum([random.randint(0, 1) for _ in range(1000)])


509

In [1]:
import pathlib
import random
from torch.utils.data import DataLoader
from transformiloop.src.data.pretraining import read_pretraining_dataset
from transformiloop.src.data.spindle_trains import EquiRandomSampler, SpindleTrainDataset, read_spindle_trains_labels
from transformiloop.src.utils.configs import initialize_config


ds_dir = '/project/portiloop_transformer/transformiloop/dataset'
MASS_dir = '/project/portiloop_transformer/transformiloop/dataset/MASS_preds'
# Read all the subjects available in the dataset
labels = read_spindle_trains_labels(ds_dir) 

In [2]:
config = initialize_config("test")
config['model_type'] = "transformer"
config['classes'] = 2

# Divide the subjects into train and test sets
subjects = list(labels.keys())
# random.shuffle(subjects)
train_subjects = [subjects[0]]

# Read the pretraining dataset
data = read_pretraining_dataset(MASS_dir, patients_to_keep=train_subjects)

# Create the train and test datasets
train_dataset = SpindleTrainDataset(train_subjects, data, labels, config)

Shuffling took 0.0362246036529541 seconds
Number of spindle labels: 95264


In [3]:
import torch
torch.unique(train_dataset.full_labels)

tensor([0, 1, 2, 3], dtype=torch.uint8)

In [4]:
count = []
for index in train_dataset.spindle_labels_first:
    if train_dataset.full_labels[index] != 2:
        count.append(index)

In [5]:
# len(train_dataset.spindle_labels_train)
# torch.unique(torch.tensor(count))
count
# train_dataset.full_labels[count[0]-1]

[]

In [6]:
for index ,(onset, offset, l) in enumerate(zip(labels[train_subjects[0]]['onsets'], labels[train_subjects[0]]['offsets'], labels[train_subjects[0]]['labels_num'])):
    if onset <= count[0] <= offset:
        print(f"Found {onset} {offset} {l}")
        print(index)

IndexError: list index out of range

In [98]:
labels[train_subjects[0]]['labels_num'][133]

3

In [99]:
1412250 / 250 * 256

1446144.0